# Semantic Heterogeneous Database Simulations

Let's start generating random records and semantic operations, which will be used to execute performance tests

In [1]:
import time
import json
import random
import math
import pandas as pd
import numpy as np
from pymongo import MongoClient
from database_generator import DatabaseGenerator
from datetime import datetime

## Load Phase

Inserting all records generated in the semantic heterogeneous database. Please note PyMongo library may diminish performance of insertions. However, because the simulator internally uses it, it is fair to also use it on our baseline test, so these delays might net. 

Let's decide all variables for this test

In [2]:
number_of_records = 100000
number_of_versions = 5
number_of_fields = 11
number_of_values_in_domain=20

number_of_tests = 5
confidence_interval = 0.95

host = 'localhost'

performance_results = pd.DataFrame()

### First scenario
Inserting all records and adding the semantic operations afterwards. 

In [3]:
def first_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    d.collection.insert_many_by_dataframe(records, 'valid_from_date')

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:    
        d.collection.execute_operation(operation[0],operation[1],operation[2])
    
    end = time.time()
    d.destroy()
    return (end - start)

In [4]:
for i in range(number_of_tests):
    time_taken = first_scenario()
    performance_results = performance_results.append({'stage' : 'LoadingPhase', 'experiment': 'insert-first','time':time_taken}, ignore_index=True)

### Second Scenario
Loading records only after inserting semantic operations in the collection

In [5]:
def second_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    d.collection.insert_many_by_dataframe(records.head(10), 'valid_from_date') #initial insert

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:          
        d.collection.execute_operation(operation[0],operation[1],operation[2])

    d.collection.insert_many_by_dataframe(records.head(-10), 'valid_from_date')
    
    end = time.time()
    d.destroy()
    return (end - start)

In [6]:
for i in range(number_of_tests):
    time_taken = second_scenario()
    performance_results = performance_results.append({'stage' : 'LoadingPhase', 'experiment': 'operations-first','time':time_taken}, ignore_index=True)

### Baseline Scenario

In a common database scenario, the records would be just inserted as they were generated (in raw format). User would have to deal with heterogeneity afterwards, in the querying fase. Therefore, for the loading phase, only generate raw records and bulk insert into the database. 

In [7]:
def third_scenario():    
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    start = time.time()
    
    client = MongoClient(host)
    client[d.database_name][d.collection_name].insert_many(d.records)
    
    end = time.time()
    d.destroy()
    return (end - start)

In [8]:
for i in range(number_of_tests):
    time_taken = third_scenario()
    performance_results = performance_results.append({'stage' : 'LoadingPhase', 'experiment': 'baseline','time':time_taken}, ignore_index=True)

## Querying tests

For the query tests, it does not matter which database (from first loading phase or from the second) is used. Both of them posess the same number of records, fields and domain values. Let's now analyse statistics in six different scenarios, just as in YCDB benchmark.

In [9]:
def generate_test_database_preinsert():
    ### Generate database just as before
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    
    d.collection.insert_many_by_dataframe(records, 'valid_from_date')

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:    
        d.collection.execute_operation(operation[0],operation[1],operation[2])  
       
    
    return d

In order to correctly compare performance between the developed system and the baseline (an ordinary document-oriented database), this test performs a query rewriting before starting the time counter. It is important to notice, however, this test compares time performance only, regardless of usability gains achieved by this system. This query rewriting would have to be manually performed by the user, which would potentially also spend time.

In [10]:
def rewrite_query_baseline(query, database_generator):
    ors = list()

    for operation in database_generator:
        operationType, version_date, arguments = operation

        if operationType == 'grouping':
            pass
        elif operationType == 'translation':
            oldValue = arguments['from']
            newValue = arguments['to']
            field = arguments['field']

            if field not in query:
                continue
            else:
                ands = [{field : newValue}]
                

        else:
            raise BaseException('OperationType unknown')

In [11]:
def generate_test_database_postinsert():
    ### Generate database just as before
    d = DatabaseGenerator()
    d.generate(number_of_records=number_of_records, number_of_versions=1, number_of_fields=number_of_fields,number_of_values_in_domain=number_of_values_in_domain)
    records = pd.DataFrame(d.records)

    
    d.collection.insert_many_by_dataframe(records.head(10), 'valid_from_date') #initial insert

    for i in range(4):
        d.generate_version()        
    
    for operation in d.operations:          
        d.collection.execute_operation(operation[0],operation[1],operation[2])

    d.collection.insert_many_by_dataframe(records.head(-10), 'valid_from_date')    
    
    return d    

In [12]:
def update_and_read_test(percent_of_update):
    ### Generate database just as before    
    d = generate_test_database_preinsert()   
    original_records = d.records.copy()

    updates = math.floor(100*percent_of_update)
    reads = 100-updates

    sequence = ([True]*updates)
    sequence.extend([False]*reads)
    random.shuffle(sequence)    

    records = [d.generate_record() for i in range(updates)]
    records_2 = records.copy()

    queries = []   

    for i in range(reads):        
        field = (random.choice(d.fields))[0]
        value = random.choice(d.field_domain[field])
        queries.append({field:value})   

    queries_2 = queries.copy()     

    start = time.time()
    for operation in sequence:             
        if operation: # insert - Nos nossos casos de uso não faz muito sentido deleções e updates. 
            record = records.pop()
            d.collection.insert_one(json.dumps(record, default=str),record['valid_from_date'])            
        else:
            d.collection.find_many(queries.pop()) ##Nao to considerando a presença ou ausência de índices
        
    end = time.time()
    preinsered_time = end-start
    d.destroy()

    client = MongoClient(host)        
    db = client[d.database_name]
    base_collection = db[d.collection_name]

    base_collection.insert_many(original_records)

    start = time.time()    
    for operation in sequence:             
        if operation: 
            record = records_2.pop()
            base_collection.insert_one(record)            
        else:
            base_collection.find(queries_2.pop()) ##Isso nao faz exatamente sentido. Deveria gerar uma nova query 
    end = time.time()    
    baseline_time = (end-start)
    client.drop_database(d.database_name)
    
    return ({'preinserted': preinsered_time, 'baseline': baseline_time})
    

### Scenario 1 - 50/50

In the first update scenario, a workload of 50% of reads and 50% of writes.

Note there is a difference between the test using the first generation method and the second one, due to lazy records evolution adopted in the prototype.

In [13]:
for i in range(number_of_tests):
    time_taken = update_and_read_test(0.5)
    performance_results = performance_results.append({'stage' : '50/50', 'experiment': 'baseline','time':time_taken['baseline']}, ignore_index=True)
    performance_results = performance_results.append({'stage' : '50/50', 'experiment': 'insert-first','time':time_taken['preinserted']}, ignore_index=True)

versions read: 0.0027930736541748047
Raw Insertion:0.002568483352661133
Processed Insertion: 0.005379199981689453
Fields Loop:0.03186607360839844
versions read: 0.0027174949645996094
Raw Insertion:0.0021827220916748047
Processed Insertion: 0.0032372474670410156
Fields Loop:0.01900625228881836
versions read: 0.001580953598022461
Raw Insertion:0.0015146732330322266
Processed Insertion: 0.0016713142395019531
Fields Loop:0.021131515502929688
versions read: 0.0026121139526367188
Raw Insertion:0.0018885135650634766
Processed Insertion: 0.0013289451599121094
Fields Loop:0.025129318237304688
versions read: 0.0015861988067626953
Raw Insertion:0.0018525123596191406
Processed Insertion: 0.0011742115020751953
Fields Loop:0.01611018180847168
versions read: 0.00286865234375
Raw Insertion:0.0029060840606689453
Processed Insertion: 0.0017805099487304688
Fields Loop:0.022131919860839844
versions read: 0.0014910697937011719
Raw Insertion:0.0013477802276611328
Processed Insertion: 0.0018112659454345703
F

KeyboardInterrupt: 

### Scenario 2 - Read Heavy

In this scenario, 95% of operations executed are read operations, while 5% are write operations (insertion).

In [27]:
for i in range(number_of_tests):
    time_taken = update_and_read_test(0.05)
    performance_results = performance_results.append({'stage' : 'ReadHeavy', 'experiment': 'baseline','time':time_taken['baseline']}, ignore_index=True)
    performance_results = performance_results.append({'stage' : 'ReadHeavy', 'experiment': 'insert-first','time':time_taken['preinserted']}, ignore_index=True)

### Scenario 3 - WriteHeavy

In this scenario, 95% of operations executed are write operations, while 5% are read operations.

In [31]:
for i in range(number_of_tests):
    time_taken = update_and_read_test(0.95)
    performance_results = performance_results.append({'stage' : 'WriteHeavy', 'experiment': 'baseline','time':time_taken['baseline']}, ignore_index=True)
    performance_results = performance_results.append({'stage' : 'WriteHeavy', 'experiment': 'insert-first','time':time_taken['preinserted']}, ignore_index=True)

In [32]:
performance_results

,stage,experiment,time
0,LoadingPhase,insert-first,32.817153
1,LoadingPhase,insert-first,35.186351
2,LoadingPhase,insert-first,25.698437
3,LoadingPhase,insert-first,30.748546
4,LoadingPhase,insert-first,29.375622
5,LoadingPhase,operations-first,9.976629
6,LoadingPhase,operations-first,9.816752
7,LoadingPhase,operations-first,10.299915
8,LoadingPhase,operations-first,9.781317
9,LoadingPhase,operations-first,14.233770
